# Database for Scientific Knowledge 

>  A local Postgresql database of scientific content with associated information generated by the agent. The intent is to use this repository as 'memory' in a Langchain-enabled agent.

In [4]:
#| default_exp utils.ceifns_db

In [6]:
#| hide
from nbdev import *

We define a local database for use with the Alhazen schema conforming to the following schema:

![Schema](pics/schema/Alhazen_Schema_v0.0.2.png) 

The key elements of the schema are:

**Abstract Parent Class**

* *InformationContentEntity* - a named piece of information (all other entities inherit from this)
  
**Classes Denoting Extant Scientific Knowledge from External Sources**

* *ScientificKnowledgeCollection* - a collection of scientific knowledge expressions
* *ScientificKnowledgeExpression* - an expression of scientific knowledge (e.g., a paper, a database record, a blog post, etc.) 
* *ScientificKnowledgeItem* - the data item that manifests the expression (e.g., a pdf, an html data file, a yaml file, etc.) 
* *ScientificKnowledgeFragment* - a relevant fragment of the item (e.g., a paragraph, a table, a figure, etc.)            

**Classes Denoting Information Generated by Alhazen**

* *Note* - an annotation of the fragment (e.g., a comment, a question, a link to another fragment, etc.)

> Our definitions are inspired by [FABIO](http://www.sparontologies.net/ontologies/fabio), but only differentiate between '*expression*' (as the reference to an scientific underlying work product, e.g., a paper or database record) and '*item*' (as the actual data that delivers on that expression). 
> The system is coded in LinkML and generated as a Postgresql database and a SQLAlchemy ORM.

In [3]:
#| export

import local_resources.linkml as linkml

from alhazen.utils.airtableUtils import AirtableUtils
from alhazen.utils.searchEngineUtils import ESearchQuery, EuroPMCQuery
from alhazen.utils.pdf_research_article_text_extractor import LAPDFBlockLoader, HuridocsPDFLoader

from alhazen.utils.queryTranslator import QueryTranslator, QueryType
from alhazen.schema_sqla import ScientificKnowledgeCollection, \
    ScientificKnowledgeExpression, ScientificKnowledgeExpressionXref, ScientificKnowledgeCollectionHasMembers, \
    ScientificKnowledgeItem, ScientificKnowledgeExpressionHasRepresentation, \
    ScientificKnowledgeFragment, ScientificKnowledgeItemHasPart, \
    InformationResource, Note, NoteIsAbout
import alhazen.schema_python as linkml_py
from alhazen.utils.jats_text_extractor import NxmlDoc

from dataclasses import dataclass, field
from datetime import datetime
from importlib_resources import files
from langchain.pydantic_v1 import BaseModel, Extra, Field, root_validator
import local_resources.linkml as linkml
import nltk.data
import os
import pandas as pd
from pathlib import Path
import re
from sqlalchemy import create_engine, exists, Engine
from sqlalchemy.orm import sessionmaker, Session
import sqlite3  
import sys
from time import time,sleep
from tqdm import tqdm
import yaml
import uuid
from langchain.vectorstores.pgvector import PGVector


In [ ]:
#| export

@dataclass
class QuerySpec:
    
    """Class that permits definition of a query specification for a literature database ."""
    name: str
    id_col: str
    query_col: str
    name_col: str
    col_map: dict[str, str] = field(default_factory=dict)
    sections: list[str] = field(default_factory=list)

    def process_cdf(self, cdf: pd.DataFrame):
        cdf = cdf.rename(columns={self.id_col:'ID', self.query_col:'QUERY'})
        cdf = cdf.rename(columns=self.col_map)
        cdf = cdf.fillna('').rename(
            columns={c:re.sub('[\s\(\)]','_', c.upper()) for c in cdf.columns}
            )
        cdf.QUERY = [re.sub('^http[s]*://', '', r.QUERY) if r.QUERY[:4]=='http' else r.QUERY 
                     for i,r in cdf.iterrows()] 
        cdf.QUERY = [re.sub('/$', '', r.QUERY.strip())  
                        for i,r in cdf.iterrows()]
        return cdf

class Ceifns_LiteratureDb(BaseModel):
  """This class runs a set of queries on external literature databases to build a local database of linked corpora and papers.

  Functionality includes:

    * Executes queries over European PMC 
    * Can run combinatorial sets of queries using a dataframe structure
        * Requires a column of queries expressed in boolean logic
        * Optional to define a secondary spreadsheet with a column of subqueries expressed in boolean logic
    * Has capability to run boolean logic over sources (currently only European PMC, but possibly others) 
    * Builds a local Postgresql database with tables for collections, expressions, items, fragments, and notes. 
    * Provides an API for querying the database and returning results as sqlAlchemy objects.
    * Permits user to download a local copy of full text papers in NXML(JATS), PDF, and HTML format.
  """
  loc: str = Field()
  name: str = Field()
  engine: Engine = Field(default=None, init=False)
  session: Session = Field(default=None, init=False)
  sent_detector: nltk.tokenize.punkt.PunktSentenceTokenizer = Field(default=None, init=False)

  class Config:
    """Configuration for this pydantic object."""
    arbitrary_types_allowed = True

  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    
    if self.loc[-1] != '/':
      self.loc = self.loc + '/' 
    if os.path.exists(self.loc) is False:
      os.makedirs(self.loc)

    db_dir = Path(self.loc + self.name)
    if db_dir.exists() is False:
      os.makedirs(db_dir)

    self.engine = create_engine('postgresql+psycopg2:///'+self.name)

    self.start_session() # instantiate session 

    log_path = '%s%s_log.txt'%(self.loc, self.name)
    if os.path.exists(log_path) is False:
      Path(log_path).touch()
    
    self.sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

  def start_session(self):
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    return self.session

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Add Collections, Expressions, Items, and Fragments
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def add_full_text_for_expression(self, e,  
                                   get_nxml=True, 
                                   get_pdf=False, 
                                   get_html=False):
    """Adds Full Text ScientificKnowledgeItem data to given ScientificKnowledgeExpression if full text is already downloaded in the `{loc}/ft/{doi}` directory."""

    doi = read_information_content_entity_iri(e, 'doi')
    if doi is None:
      return  
    
    # Check local directory for nxml file
    base_file_path = '%s%s/ft/'%(self.loc, self.name)
    if get_nxml:
      nxml_file_path = base_file_path+doi+'.nxml'
      if os.path.exists(nxml_file_path):
        with open(nxml_file_path, 'r') as f:
          xml = f.read()
          try:
            d = NxmlDoc(doi, xml)
          except Exception as ex:
            return
          doc_text = d.text 
          if 'body' in [so.element.tag for so in d.standoffs]:
            ski_type = 'JATSFullText'
          else:
            ski_type = 'CitationRecord'     
          ski_id = str(uuid.uuid4().hex)[:10]
          ski = ScientificKnowledgeItem(id=ski_id, 
                                        content=doc_text,
                                        xref=['file:'+nxml_file_path],
                                        type=ski_type,)
          e.has_representation.append(ski)
          self.session.add(ski)
          self.session.flush()
          self.add_sections_as_fragments_from_nxmldoc(ski, d)

    # Check local directory for pdf file
    if get_pdf:
      pdf_path = base_file_path+doi+'.pdf'
      loader = HuridocsPDFLoader(pdf_path)
      docs = loader.load(curi_id='doi:'+doi) 
      skfs = []
      doc_text = ''
      ski_id = str(uuid.uuid4().hex)[:10]
      for i, d in enumerate(docs):
        sentence_split_text = '\n'.join(self.sent_detector.tokenize(d.page_content))

        skf = ScientificKnowledgeFragment(id=ski_id+'.'+str(i), 
                                             type='section',
                                             offset=len(doc_text),
                                             length=len(sentence_split_text),
                                             name=sentence_split_text.split('\n')[0],
                                             content=sentence_split_text)
        if len(doc_text) > 0:
          doc_text += '\n\n'
        doc_text += sentence_split_text
        self.session.add(skf)
        self.session.flush()
        skfs.append(skf)
      ski_type = 'PDFFullText'
      ski = ScientificKnowledgeItem(id=ski_id, 
                                    content=doc_text,
                                    xref=['file:'+pdf_path],
                                    type=ski_type,)
      e.has_representation.append(ski)
      self.session.add(ski)
      self.session.flush()
      for skf in skfs:
        ski.has_part.append(skf)
        skf.part_of = ski.id
        self.session.flush()
        
    if get_html:
      placeholder = 1  

  def add_sections_as_fragments_from_nxmldoc(self, item, d):
    """Adds fragments from an NxmlDoc object to a ScientificKnowledgeItem object."""

    # ['PMID', 'PARAGRAPH_ID', 'TAG', 'TAG_TREE', 'OFFSET', 'LENGTH', 'FIG_REF', 'PLAIN_TEXT'
    fragments = []
    df = d.build_enhanced_document_dataframe()
    df = df[df.TAG_TREE!='']
    # Can search for a substring in top section headers
    # df3 = df[df.TOP_SECTION.str.contains('method', case=False)]

    df3 = df[df.TOP_SECTION!='']
    df4 = df3.groupby('SECTION_TREE').agg({'OFFSET': 'min', 'LENGTH': 'sum', 'PLAIN_TEXT':lambda x: '\n'.join(x)}).sort_values('OFFSET')
    df4 = df4.reset_index(drop=False)
    
    if df4 is None:
      return
    
    for i, tup in df4.iterrows():      
      sentence_split_text = '\n'.join(self.sent_detector.tokenize(tup.PLAIN_TEXT))
      fragment = ScientificKnowledgeFragment(id=item.id[:10]+'.'+str(i), 
                                             type='section',
                                             offset=tup.OFFSET,
                                             length=tup.LENGTH,
                                             name=tup.SECTION_TREE,
                                             content=sentence_split_text)
      self.session.add(fragment)
      self.session.flush()
      item.has_part.append(fragment)
      fragment.part_of = item.id
      fragments.append(fragment)
      self.session.flush()     

  def add_full_text_for_collection(self, collection_id, 
                                   get_nxml=True, get_pdf=True, get_html=True):
    for e in self.list_expressions(collection_id=collection_id):
      doi = read_information_content_entity_iri(e, 'doi')
      if doi is None:
        continue  
      self.add_full_text_for_expression(e, get_nxml, get_pdf, get_html)      

  def add_corpus_from_epmc(self, qt, qt2, 
                           sections=['paper_title', 'ABSTRACT'], 
                           sections2=['paper_title', 'ABSTRACT'], 
                           page_size=1000):
    """Adds corpora based on coupled QueryTranslator objects."""
    
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()

    if self.session.query(exists().where(InformationResource.name=='EPMC')).scalar():
      info_resource = self.session.query(InformationResource) \
          .filter(InformationResource.name=='EPMC').first()
    else:
      info_resource = InformationResource(id='skem:EPMC', 
                                          iri=['skem:EPMC'], 
                                          name='European PubMed Central', 
                                          type='skem:InformationResource',
                                          xref=['https://europepmc.org/'])

    (corpus_ids, epmc_queries) = qt.generate_queries(QueryType.epmc, sections=sections)
    if qt2:
      (subset_ids, epmc_subset_queries) = qt2.generate_queries(QueryType.epmc, sections=sections2)
    else: 
      (subset_ids, epmc_subset_queries) = ([0],[''])
    for (i, q) in zip(corpus_ids, epmc_queries):
      for (j, sq) in zip(subset_ids, epmc_subset_queries):
        query = q
        corpus_id = str(i)
        corpus_name = qt.df.loc[qt.df['ID']==i][qt.name_col].values[0]
        if query is None or query=='nan' or len(query)==0: 
          continue
        if len(sq) > 0:
          query = '(%s) AND (%s)'%(q, sq)
          if j is not None:
            corpus_id = str(i)+'.'+str(j)
            corpus_name2 = qt2.df.loc[qt2.df['ID']==j][qt2.name_col].values[0]
            corpus_name = corpus_name + '/'+ corpus_name2
        self.add_corpus_from_epmc_query(corpus_id, corpus_name, query)
    self.session.commit()
  
  def add_corpus_from_epmc_query(self, 
                                 corpus_id, 
                                 corpus_name, 
                                 query, 
                                 commit_this=True,
                                 page_size=1000):
    """Adds corpus to database based on terms constructed in a direct EPMC query."""
    
    # does this collection already exist?  
    corpus_id = str(corpus_id)
    if self.session.query(exists().where(ScientificKnowledgeCollection.id==corpus_id)).scalar():
      raise Exception('Collection already exists')

    info_resource = InformationResource(id='skem:EPMC', 
                                          iri=['skem:EPMC'], 
                                          name='European PubMed Central', 
                                          type='skem:InformationResource',
                                          xref=['https://europepmc.org/'])
    if self.session.query(exists().where(InformationResource.name=='EPMC')).scalar():
      info_resource = self.session.query(InformationResource) \
          .filter(InformationResource.name=='EPMC').first()
    
    corpus = ScientificKnowledgeCollection(id=corpus_id,
                                           type='skem:ScientificKnowledgeCollection',
                                           provenance=[query], 
                                           name=corpus_name,
                                           has_members=[])
    self.session.add(corpus)

    epmcq = EuroPMCQuery()
    numFound, pubs = epmcq.run_empc_query(query, page_size=page_size)
    for p in tqdm(pubs):
      p_id = str(p.id)
      p_check = self.session.query(ScientificKnowledgeExpression) \
          .filter(ScientificKnowledgeExpression.id==p_id).first()
      if p_check is not None:
        p = p_check
      corpus.has_members.append(p)
      for item in p.has_representation:
        for f in item.has_part:          
          f.part_of = item.id
          self.session.add(f)
        item.represented_by = p.id
        self.session.add(item)
      self.session.add(p)
    if commit_this:
      self.session.commit()

  def delete_collection(self, collection_id, commit_this=True):    
    """Deletes the specified collection from the database, leave all expressions and items intact"""  

    if self.session.query(exists().where(ScientificKnowledgeCollection.id==collection_id)).scalar():
      cp = self.session.query(ScientificKnowledgeCollection) \
          .join(ScientificKnowledgeCollection.has_members, isouter=True) \
          .filter(ScientificKnowledgeCollection.id==collection_id).first()
      if cp is not None:
        for p in cp.has_members:
          cp.has_members.remove(p)
        self.session.flush()
        self.session.delete(cp)
      if commit_this:
        self.session.commit()
    else:
      raise Exception('Collection does not exist')

  def check_query_terms(self, qt, qt2=None, pubmed_api_key=''):
    pmq = ESearchQuery(api_key=pubmed_api_key)
    terms = set()
    for t in qt.terms2id.keys():
        terms.add(t)
    if qt2 is not None:
        for t2 in qt2.terms2id.keys():
            terms.add(t2)
    check_table = {} 
    for t in tqdm(terms):
        (is_ok, t2, c) = pmq._check_query_phrase(t)
        check_table[t] = (is_ok, c)
    return check_table

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Query Collections, Expressions, Items, and Fragments
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

  def get_expression_by_doi(self, doi):
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    """Returns a ScientificKnowledgeExpression object for a given DOI."""
    query = self.session.query(ScientificKnowledgeExpression) \
        .filter(ScientificKnowledgeExpression.id==ScientificKnowledgeExpressionXref.ScientificKnowledgeExpression_id) \
        .filter(ScientificKnowledgeExpressionXref.xref=='doi:'+doi)
    ske = query.first()
    return ske

  def list_collections(self, search_term=None):
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    if search_term:
      q = (self.session.query(ScientificKnowledgeCollection) 
          .filter(ScientificKnowledgeCollection.name == search_term))
    else:
      q = self.session.query(ScientificKnowledgeCollection)
    for c in q.all():
      yield(c)

  def list_expressions(self, collection_id=None, search_term=None): 
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    if collection_id:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(ScientificKnowledgeCollectionHasMembers,
                              ScientificKnowledgeExpression,
                              ScientificKnowledgeExpressionHasRepresentation,
                              ScientificKnowledgeItem) 
            .filter(ScientificKnowledgeCollectionHasMembers.ScientificKnowledgeCollection_id==collection_id)
            .filter(ScientificKnowledgeCollectionHasMembers.has_members_id == ScientificKnowledgeExpression.id)
            .filter(ScientificKnowledgeExpression.id == ScientificKnowledgeExpressionHasRepresentation.ScientificKnowledgeExpression_id)
            .filter(ScientificKnowledgeExpressionHasRepresentation.has_representation_id == ScientificKnowledgeItem.id)
            .filter(ScientificKnowledgeItem.content.like(search))
          )
      else:
         q = (self.session.query(ScientificKnowledgeCollection, 
                              ScientificKnowledgeCollectionHasMembers,
                              ScientificKnowledgeExpression ) 
            .filter(ScientificKnowledgeCollection.id == ScientificKnowledgeCollectionHasMembers.ScientificKnowledgeCollection_id)
            .filter(ScientificKnowledgeCollectionHasMembers.has_members_id == ScientificKnowledgeExpression.id)
            .filter(ScientificKnowledgeCollection.id == collection_id)
          )
    else:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(ScientificKnowledgeExpression,
                              ScientificKnowledgeExpressionHasRepresentation,
                              ScientificKnowledgeItem) 
            .filter(ScientificKnowledgeCollectionHasMembers.has_members_id == ScientificKnowledgeExpression.id)
            .filter(ScientificKnowledgeExpression.id == ScientificKnowledgeExpressionHasRepresentation.ScientificKnowledgeExpression_id)
            .filter(ScientificKnowledgeExpressionHasRepresentation.has_representation_id == ScientificKnowledgeItem.id)
            .filter(ScientificKnowledgeItem.content.like(search))
          )
      else:
        q = self.session.query(ScientificKnowledgeExpression)
      
    for c in q.all():
      yield(c)

  def list_notes_for_fragments_in_paper(self, run_name, paper_id, item_type='FullTextPaper'):
    '''returns notes of a specific type associated with fragments from a given paper .'''
    q1 = self.session.query(ScientificKnowledgeItem) \
            .filter(ScientificKnowledgeExpression.id == ScientificKnowledgeExpressionHasRepresentation.ScientificKnowledgeExpression_id) \
            .filter(ScientificKnowledgeExpressionHasRepresentation.has_representation_id == ScientificKnowledgeItem.id) \
            .filter(ScientificKnowledgeItem.type == item_type) \
            .filter(ScientificKnowledgeExpression.id.like('%'+str(paper_id)+'%')) 
    i = q1.first()
    if i is None:
      return []
    for f in i.has_part:
      for n in f.has_notes:
        if n.name == run_name:
          yield(n)
         
  def list_fragments_for_paper(self, paper_id, item_type):
    '''Loads fragments from a given paper sections of a specified paper from the local database.'''
    q1 = self.session.query(ScientificKnowledgeItem) \
            .filter(ScientificKnowledgeExpression.id == ScientificKnowledgeExpressionHasRepresentation.ScientificKnowledgeExpression_id) \
            .filter(ScientificKnowledgeExpressionHasRepresentation.has_representation_id == ScientificKnowledgeItem.id) \
            .filter(ScientificKnowledgeItem.type == item_type) \
            .filter(ScientificKnowledgeExpression.id.like('%'+str(paper_id)+'%')) 
    i = q1.first()
    if i is None:
        raise Exception('No items of format: %s found for expression: %s'%(item_type, str(paper_id)))
    fragments = []
    for f in i.has_part:
      fragments.append(f)
    return sorted(fragments, key=lambda f: f.offset)
  
  def list_fragments(self, expression_id=None, search_term=None):
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    if expression_id:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(ScientificKnowledgeExpressionHasRepresentation,
                              ScientificKnowledgeItem,
                              ScientificKnowledgeItemHasPart,
                              ScientificKnowledgeFragment) 
            .filter(ScientificKnowledgeExpressionHasRepresentation.ScientificKnowledgeExpression_id == expression_id)
            .filter(ScientificKnowledgeExpressionHasRepresentation.has_representation_id == ScientificKnowledgeItem.id)
            .filter(ScientificKnowledgeItem.id == ScientificKnowledgeItemHasPart.ScientificKnowledgeItem_id)
            .filter(ScientificKnowledgeItemHasPart.has_part_id == ScientificKnowledgeFragment.id)
            .filter(ScientificKnowledgeFragment.content.ilike('%'+search+'%'))
          )
      else:
        q = (self.session.query(ScientificKnowledgeExpressionHasRepresentation,
                              ScientificKnowledgeItem,
                              ScientificKnowledgeItemHasPart,
                              ScientificKnowledgeFragment) 
            .filter(ScientificKnowledgeExpressionHasRepresentation.ScientificKnowledgeExpression_id == expression_id)
            .filter(ScientificKnowledgeExpressionHasRepresentation.has_representation_id == ScientificKnowledgeItem.id)
            .filter(ScientificKnowledgeItem.id == ScientificKnowledgeItemHasPart.ScientificKnowledgeItem_id)
            .filter(ScientificKnowledgeItemHasPart.has_part_id == ScientificKnowledgeFragment.id)
          )
    else:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(ScientificKnowledgeExpressionHasRepresentation,
                              ScientificKnowledgeItem,
                              ScientificKnowledgeItemHasPart,
                              ScientificKnowledgeFragment) 
            .filter(ScientificKnowledgeExpressionHasRepresentation.has_representation_id == ScientificKnowledgeItem.id)
            .filter(ScientificKnowledgeItem.id == ScientificKnowledgeItemHasPart.ScientificKnowledgeItem_id)
            .filter(ScientificKnowledgeItemHasPart.has_part_id == ScientificKnowledgeFragment.id)
            .filter(ScientificKnowledgeFragment.content.ilike('%'+search+'%'))
          )
      else:
        q = self.session.query(ScientificKnowledgeFragment)
    for c in q.all():
      yield(c)

In [1]:
#| export

def read_information_content_entity_iri(ice, id_prefix):
    """Reads an identifier for a given prefix"""
    idmap = {k[:k.find(':')]:k[k.find(':')+1:] for k in ice.xref} 
    return idmap.get(id_prefix)